In [4]:
# %pip install pantab
# %pip install numpy --upgrade
# %pip install tableau_api_lib
# %pip install tableauserverclient

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for tableauserverclient: filename=tableauserverclient-0.19.0-py2.py3-none-any.whl size=121863 sha256=760aef32e8566a48166c151f6b297bd7e1a540461d179a467d3a3f8dbd3cfe64
  Stored in directory: c:\users\abc\appdata\local\pip\cache\wheels\57\86\43\5116ec8345ba60224132d4f4ab0a56b0d6ae9a3f25405c33c1
Successfully built tableauserverclient
Note: you may need to restart the kernel to use updated packages.


In [6]:
   
#===================================================================
#Title: Publish Tableau Hyper Extract
#Authors: 
#Date: 2022-08-09
#Description: The code below shows an example of publishing a dataframe to Tableau using the Hyper API.
#===================================================================
import pandas as pd
# import hvac
import sqlalchemy
import urllib
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from datetime import datetime

#Used for database
import pyodbc

#Used for publishTableau
import pantab
from tableauhyperapi import HyperProcess, Connection, TableDefinition, SqlType, Telemetry, Inserter, CreateMode, TableName
from tableauhyperapi import escape_string_literal
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils.querying import get_projects_dataframe
import tableauserverclient as TSC
import os
from pathlib import Path

In [16]:
#Script configuration. Generate a UUID and enter it below.
taskID = '001'
taskName = 'PublishTableau - Test'
logID = '' #Default, use for DEV
#logID = '' #Use this one for PRODUCTION
startTime = datetime.now()


#===================================================================
#Standard log writer
#Logging function
def writeLog(taskStatus, taskMessage, endTime = None):
    log_engine = connectSQLServer1()
    if endTime == None:
        endTime = datetime.now()
    logColumns = ['TaskID', 'TaskName','Status','Message','StartTime','EndTime']
    df = pd.DataFrame([[taskID, taskName, taskStatus, taskMessage, startTime, endTime]], columns=logColumns)
    df.to_sql('Python_Logging', log_engine, if_exists='append', index=False)

#===================================================================    
#Connection Functions
#SQL Server Connection

def connectSQLServer1():
    SQLConn = "DRIVER={SQL Server Native Client 11.0};""SERVER=localhost\SQLEXPRESS;""DATABASE=datahub;""UID=sa;""PWD=user1"
    SQLparams = urllib.parse.quote_plus(SQLConn)
    SQLEngine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % SQLparams)
    return SQLEngine


#Tableau Connection - Requires pantab, tableauhyperapi, tableau_api_lib, tableauserverclient
def publishTableau(siteName, projectName, name, df):
    #Save DataFrame output to .hyper file locally
    fileName = str(Path( "__file__" ).parent.absolute())+ '\\' + name + '.hyper'
    pantab.frame_to_hyper(df, fileName, table = name)
    
    #Connect to our Tableau Instance
    tableau_auth = TSC.PersonalAccessTokenAuth('user2', 'Wc+HPIMURkK0piACTSMSaw==:cxg6XI0vOfwWrlmPUu8LRBphobfOEwva', siteName)
    server = TSC.Server('https://prod-ca-a.online.tableau.com/')
    server.add_http_options({'verify': False})
    server.use_server_version()

    with server.auth.sign_in(tableau_auth):
        #Define publish mode - Overwrite, Append, or CreateNew
        publishMode = TSC.Server.PublishMode.Overwrite
        
        #Get project_id from project_name
        all_projects, pagination_item = server.projects.get()
        for project in TSC.Pager(server.projects):
            if project.name == projectName:
                project_id = project.id
    
        #Create the datasource object with the project_id
        datasource = TSC.DatasourceItem(project_id)
        
        #Publish datasource
        print(f'{datetime.now()}: Publishing {name} to {projectName}...')
        datasource = server.datasources.publish(datasource, fileName, publishMode)
        print(f'{datetime.now()}: Datasource published. Datasource ID: {format(datasource.id)}')
    
    if os.path.exists(fileName):
        os.remove(fileName)
        print(f'{datetime.now()}: Removed file {fileName} from local')

#===============================================================
try:
    print(f'{startTime}: Script starting.')

    #Example Teradata Connection
    engine = connectSQLServer1()
    queryText2 =f"""
        SELECT * from UCI   
    """
    outputdata = pd.read_sql(queryText2, engine)
    print(outputdata)

    #Example Publish dataframe to Tableau
    publishTableau('coop', 'Coop', 'test3', outputdata)

    #Write a "pass" status to the log (uncomment writeLog after GUID is entered):
    taskMessage = 'Logging Template Pass'
    writeLog(True, taskMessage) 
    print(f'{datetime.now()}: Done!')

except Exception as e:
    #Write a "fail" status to the log (uncomment writeLog after GUID is entered):
    taskMessage = str(e)
    print(taskMessage)
    #writeLog(False, taskMessage) 
#===================================================================

2022-08-23 20:45:30.443674: Script starting.
                                    Loan ID  \
0      f738779f-c726-40dc-92cf-689d73af533d   
1      6dcc0947-164d-476c-a1de-3ae7283dde0a   
2      f7744d01-894b-49c3-8777-fc6431a2cff1   
3      83721ffb-b99a-4a0f-aea5-ef472a138b41   
4      08f3789f-5714-4b10-929d-e1527ab5e5a3   
...                                     ...   
10348                                  None   
10349                                  None   
10350                                  None   
10351                                  None   
10352                                  None   

                                Customer ID  Current Loan Amount        Term  \
0      ded0b3c3-6bf4-4091-8726-47039f2c1b90             611314.0  Short Term   
1      1630e6e3-34e3-461a-8fda-09297d3140c8             266662.0  Short Term   
2      2c60938b-ad2b-4702-804d-eeca43949c52             153494.0  Short Term   
3      12116614-2f3c-4d16-ad34-d92883718806             176242.0  Shor